# Table Generation

## Overview
Demonstrate the table generation system: number formatting, colour-coded indicators, executive and technical tables, decision matrices, and export to multiple formats.

- **Prerequisites**: [getting_started/01_introduction](../getting_started/01_introduction.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Practitioner]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reporting'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.reporting.table_generator import TableGenerator, NumberFormatter, ColorCoder

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Number Formatting

In [ ]:
formatter = NumberFormatter()

print("Currency:")
for v in [1234.56, 1_234_567, -50_000]:
    print(f"  {v:>12,.2f}  ->  {formatter.format_currency(v)}")

print("\nPercentage:")
for v in [0.1523, -0.042, 0.0008]:
    print(f"  {v:>8.4f}  ->  {formatter.format_percentage(v)}")

print("\nLarge numbers (abbreviated):")
for v in [5e6, 1.2e9, 3.45e12]:
    print(f"  {v:>16.0f}  ->  {formatter.format_number(v, abbreviate=True)}")

## 2. Colour-Coded Indicators

In [ ]:
html_coder = ColorCoder(output_format="html")
terminal_coder = ColorCoder(output_format="terminal")

metrics = [
    ("ROE", 0.18, 0.10, 0.15),
    ("Ruin Probability", 0.008, 0.05, 0.02),
    ("Growth Rate", 0.045, 0.03, 0.06),
]

print("Terminal colour codes:")
for name, value, lo, hi in metrics:
    thresholds = {
        "good": (hi, None),
        "warning": (lo, hi),
        "bad": (None, lo),
    }
    indicator = terminal_coder.traffic_light(value, thresholds)
    print(f"  {name:20s}: {value:.3f}  {indicator}")

## 3. Executive Tables

### Optimal Insurance Limits by Company Size

In [ ]:
generator = TableGenerator(default_format="markdown")

data = pd.DataFrame({
    "Company Size": ["$1M", "$10M", "$50M", "$100M"],
    "Primary Limit": ["$500K", "$2M", "$5M", "$10M"],
    "Excess Limit": ["$2M", "$10M", "$25M", "$50M"],
    "Optimal Deductible": ["$25K", "$100K", "$250K", "$500K"],
    "Expected ROE": ["12.5%", "15.2%", "16.8%", "17.1%"],
    "Ruin Prob.": ["2.1%", "0.8%", "0.3%", "0.1%"],
})

print(generator.generate(data, caption="Optimal Insurance Limits by Company Size"))

### Decision Matrix

In [ ]:
alternatives = ["No Insurance", "Basic Coverage", "Optimal Structure", "Full Coverage"]
criteria = ["ROE", "Risk Score", "Cost Efficiency", "Implementation Ease"]
scores = np.array([
    [0.12, 0.30, 0.90, 0.90],
    [0.14, 0.60, 0.70, 0.80],
    [0.185, 0.85, 0.65, 0.60],
    [0.15, 0.95, 0.40, 0.50],
])
weights = [0.4, 0.3, 0.2, 0.1]

print(generator.generate_decision_matrix(
    alternatives=alternatives, criteria=criteria,
    scores=scores, weights=weights,
))

## 4. Technical Tables

In [ ]:
params = pd.DataFrame({
    "Parameter": ["Mean Return", "Volatility", "Tax Rate",
                  "Loss Frequency", "Loss Severity Mean", "Severity CV"],
    "Value": ["8.0%", "15.0%", "25.0%", "5.3/yr", "$500K", "1.5"],
    "Range Tested": ["5-12%", "10-25%", "20-35%",
                     "3-8/yr", "$200K-$1M", "0.8-2.5"],
    "Source": ["Historical", "Historical", "Tax code",
              "Claims data", "Claims data", "Fitted"],
})
print(generator.generate(params, caption="Simulation Parameters"))

## 5. Export to Multiple Formats

In [ ]:
sample = pd.DataFrame({
    "Company Size": ["$1M", "$10M", "$100M"],
    "ROE": ["12.5%", "15.2%", "17.1%"],
    "Ruin Prob.": ["2.1%", "0.8%", "0.1%"],
})

for fmt in ["markdown", "grid", "html"]:
    print(f"\n--- {fmt.upper()} ---")
    print(generator.generate(sample, output_format=fmt))

## Key Takeaways

- `NumberFormatter` handles currency, percentages, and large-number abbreviation via `format_currency`, `format_percentage`, and `format_number`.
- `ColorCoder` produces traffic-light indicators for terminal or HTML output.
- `TableGenerator` renders DataFrames in markdown, grid, or HTML and supports decision matrices.
- Tables can be embedded directly into executive or technical reports.

## Next Steps

- [reporting/02_excel_reporting](02_excel_reporting.ipynb) -- Excel financial statements
- [reporting/03_report_generation](03_report_generation.ipynb) -- automated executive and technical reports